# Анализ базы данных

---
**Основная задача проекта:** проанализировать базу данных.

---

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

### Задания

- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': ,  #пароль
 'host': ,
 'port': , # порт подключения
 'db': } # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [2]:
pd.io.sql.read_sql('SELECT * FROM books', con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
...,...,...,...,...,...,...
995,996,571,Wyrd Sisters (Discworld #6; Witches #2),265,2001-02-06,147
996,997,454,Xenocide (Ender's Saga #3),592,1996-07-15,297
997,998,201,Year of Wonders,358,2002-04-30,212
998,999,94,You Suck (A Love Story #2),328,2007-01-16,331


In [3]:
pd.io.sql.read_sql('SELECT * FROM authors', con = engine)[:5]

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
pd.io.sql.read_sql('SELECT * FROM publishers', con = engine)[:5]

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
pd.io.sql.read_sql('SELECT * FROM ratings', con = engine)[:5]

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
pd.io.sql.read_sql('SELECT * FROM reviews', con = engine)[:5]

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Книги, вышедшие после 1 января 2000 года

In [7]:
query_01 = '''
SELECT count(book_id) 
FROM books 
WHERE publication_date >= '2000-01-01'
'''


pd.io.sql.read_sql(query_01, con = engine)

,count
0,821


С 1 января 2000 года вышло 821 книга.

#### Количество обзоров и средняя оценка

In [9]:
query_02 = '''
SELECT *  FROM

(SELECT rev._book_id as id, rev._review_count as review_count, books.title as title FROM books 
LEFT JOIN (SELECT reviews.book_id as _book_id, COUNT(reviews.review_id) as _review_count FROM reviews GROUP BY reviews.book_id) rev 
ON _book_id = books.book_id) book1

LEFT JOIN (SELECT book_id, AVG(rating) FROM ratings GROUP BY ratings.book_id) rat 
    ON rat.book_id = book1.id 
    
'''

pd.io.sql.read_sql(query_02, con = engine)


,id,review_count,title,book_id,avg
0,1.0,2.0,'Salem's Lot,1.0,3.666667
1,2.0,1.0,1 000 Places to See Before You Die,2.0,2.500000
2,3.0,3.0,13 Little Blue Envelopes (Little Blue Envelope...,3.0,4.666667
3,4.0,2.0,1491: New Revelations of the Americas Before C...,4.0,4.500000
4,5.0,4.0,1776,5.0,4.000000
...,...,...,...,...,...
995,996.0,3.0,Wyrd Sisters (Discworld #6; Witches #2),996.0,3.666667
996,997.0,3.0,Xenocide (Ender's Saga #3),997.0,3.400000
997,998.0,4.0,Year of Wonders,998.0,3.200000
998,999.0,2.0,You Suck (A Love Story #2),999.0,4.500000


In [20]:
query_rev = '''

with avg_rate_table as (
select books.book_id, avg(ratings.rating) as avg_rtg
from books
join ratings on books.book_id = ratings.book_id
group by books.book_id),


count_rev_table as (
select books.book_id, count(reviews.review_id) as cnt_rev
from books
join reviews on books.book_id = reviews.book_id
group by books.book_id)


select books.book_id, books.title, avg_rate_table.avg_rtg, count_rev_table.cnt_rev
from books
left join avg_rate_table on books.book_id = avg_rate_table.book_id
left join count_rev_table on books.book_id = count_rev_table.book_id

'''

pd.io.sql.read_sql(query_rev, con = engine)

,book_id,title,avg_rtg,cnt_rev
0,652,The Body in the Library (Miss Marple #3),4.500000,2.0
1,273,Galápagos,4.500000,2.0
2,51,A Tree Grows in Brooklyn,4.250000,5.0
3,951,Undaunted Courage: The Pioneering First Missio...,4.000000,2.0
4,839,The Prophet,4.285714,4.0
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,5.000000,NaN
996,83,Anne Rice's The Vampire Lestat: A Graphic Novel,3.666667,NaN
997,221,Essential Tales and Poems,4.000000,NaN
998,387,Leonardo's Notebooks,4.000000,NaN


#### Издательство, которое выпустило наибольшее число книг толще 50 страниц

In [11]:
query_03 = '''
SELECT * FROM (SELECT publisher_id as pid, COUNT(publisher_id) 
FROM books WHERE num_pages > 50 GROUP BY publisher_id) b 
JOIN publishers ON publishers.publisher_id = b.pid 
ORDER BY count DESC

'''
pd.io.sql.read_sql(query_03, con = engine)


,pid,count,publisher_id,publisher
0,212,42,212,Penguin Books
1,309,31,309,Vintage
2,116,25,116,Grand Central Publishing
3,217,24,217,Penguin Classics
4,33,19,33,Ballantine Books
...,...,...,...,...
329,34,1,34,Ballantine Books (NY)
330,225,1,225,Plaza y Janés
331,138,1,138,HarperCollinsPublishers
332,245,1,245,Random House: Modern Library


Издательство Penguin Books выпустило самое большое количество книг толще 50 страниц(42).

#### Автор с самой высокой средней оценкой книг 

In [12]:
query_04 = '''
SELECT * FROM (SELECT author_id, AVG(avg) 
FROM (SELECT book_id, AVG(rating) 
FROM ratings 
GROUP BY ratings.book_id HAVING COUNT(*) > 50) r 
JOIN books ON r.book_id = books.book_id 
GROUP BY author_id) a 
JOIN authors ON a.author_id = authors.author_id  
ORDER BY avg DESC

'''

pd.io.sql.read_sql(query_04, con = engine)[:1]

,author_id,avg,author_id,author
0,236,4.283844,236,J.K. Rowling/Mary GrandPré


J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой.

#### Среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [13]:
query_05 = '''
SELECT AVG(count) 
FROM (SELECT t.username, COUNT(t.username) 
FROM (SELECT * FROM (SELECT * FROM (SELECT username as uname, COUNT(username) FROM ratings GROUP BY username) u 
WHERE count > 50) rat 
JOIN reviews ON rat.uname = reviews.username) t 
GROUP BY t.username) c
'''


pd.io.sql.read_sql(query_05, con = engine)[:1]

,avg
0,24.333333


В среднем пользователи, поставившие больше 50 оценок, совершают около 24 обзоров.

## Вывод

С 1 января 2000 года вышло 821 книга.

Самое продуктивное издательство Penguin Books выпустило самое большое количество книг толще 50 страниц - 42 книги. 

J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой.

В среднем пользователи, поставившие больше 50 оценок, совершают около 24 обзоров.